In [1]:
import pandas as pd
import numpy as np

In [6]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')

In [3]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 90 or orientation_z < -90:
                motion_type = "Vertical Movement"
            elif orientation_x > 90 or orientation_x < -90:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [16]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/20
3474/3474 [==============================] - 15s 4ms/step - loss: 1.8236 - accuracy: 0.6960 - val_loss: 0.6908 - val_accuracy: 0.9406
Epoch 2/20
3474/3474 [==============================] - 14s 4ms/step - loss: 1.2233 - accuracy: 0.8680 - val_loss: 0.6342 - val_accuracy: 0.9385
Epoch 3/20
3474/3474 [==============================] - 14s 4ms/step - loss: 1.0585 - accuracy: 0.8914 - val_loss: 0.5671 - val_accuracy: 0.9377
Epoch 4/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.9513 - accuracy: 0.9069 - val_loss: 0.5453 - val_accuracy: 0.9326
Epoch 5/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.8861 - accuracy: 0.9167 - val_loss: 0.5040 - val_accuracy: 0.9396
Epoch 6/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.8799 - accuracy: 0.9237 - val_loss: 0.4559 - val_accuracy: 0.9417
Epoch 7/20
3474/3474 [==============================] - 12s 4ms/step - loss: 0.8554 - accuracy: 0.9293 - val_loss: 0.5596 - val_ac

In [11]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Vertical Movement":
        predicted_activity = ["Running", "Walking", "Walking & party", "Football"]
    elif motion_type == "Horizontal Movement":
        predicted_activity = ["In vehicle", "Train", "In Bus", "On Bus stop"]
    elif motion_type == "Moving":
        predicted_activity = ["Cooking", "Meeting", "Movie", "Shopping & wearing"]
    elif motion_type == "Turning/Tilting":
        predicted_activity = ["Phone was out of the pocket (forgot)", "Took off glasses"]
    elif motion_type == "Stationary":
        predicted_activity = ["At home", "Pause", "Shop", "Work", "Eating", "Sleeping", "Watching TV", "Eating", "Picnic", "Video Games"]
    return predicted_activity

# Assuming 'pivot_data4' is your DataFrame
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [12]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,predicted_activity
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving,"[Cooking, Meeting, Movie, Shopping & wearing]"
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,"[Phone was out of the pocket (forgot), Took of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."


In [7]:
activity_types = activity_type_full['activity_type'].unique()

In [8]:
activity_types

array(['Video games', 'In computer', 'At home', 'Eat', 'Sleep',
       'Watching TV', 'In vehicle', 'Picnic ', 'Walk', 'Running',
       'Phone was out of the pocket (forgot)', 'Train', 'Cooking',
       'In bus', 'Meeting', 'Pause', 'Movie', 'Football', 'Shop',
       'Shopping& wearing', 'Walking&party', 'On bus stop',
       'Took off glasses', 'Work'], dtype=object)

In [4]:
pivot_data4_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv'

pivot_data4.to_csv(pivot_data4_pathway, index=False)

In [5]:
pivot_data4['motion_type'].value_counts()


motion_type
Stationary             156088
Turning/Tilting          8986
Moving                   7795
Vertical Movement         502
Horizontal Movement       291
Name: count, dtype: int64

This diversity of motion_types shows that the function is working. 